# 1. Environment Setup

In [14]:
%%capture
!pip install -q "protobuf<4.21.0" tqdm lime shap scikit-image

# 2. Imports & Strategy

In [15]:
import os, cv2, warnings
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess

from sklearn.cluster import KMeans
from skimage.feature import graycomatrix, graycoprops
from skimage.segmentation import mark_boundaries

from lime import lime_image
import shap
from tqdm.keras import TqdmCallback

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

strategy = tf.distribute.MirroredStrategy()
print("GPUs:", strategy.num_replicas_in_sync)

try:
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy("mixed_float16")
    print("Mixed precision enabled")
except Exception as e:
    print("Mixed precision not enabled:", e)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
GPUs: 2
Mixed precision enabled


# 3. Dataset Paths

In [16]:
BR35H_ROOT = "/kaggle/input/brain-tumor-detection"
BR35H_YES = os.path.join(BR35H_ROOT, "yes")
BR35H_NO  = os.path.join(BR35H_ROOT, "no")

NAV_ROOT = "/kaggle/input/brain-mri-images-for-brain-tumor-detection"
NAV_YES = os.path.join(NAV_ROOT, "yes")
NAV_NO  = os.path.join(NAV_ROOT, "no")

MOS_ROOT = "/kaggle/input/brain-tumor-mri-yes-or-no/Brain (y-n)/Training"
MOS_YES = os.path.join(MOS_ROOT, "yes")
MOS_NO  = os.path.join(MOS_ROOT, "no")

print("Train:", BR35H_ROOT)
print("Val  :", NAV_ROOT)
print("Test :", "/kaggle/input/brain-tumor-mri-yes-or-no")

Train: /kaggle/input/brain-tumor-detection
Val  : /kaggle/input/brain-mri-images-for-brain-tumor-detection
Test : /kaggle/input/brain-tumor-mri-yes-or-no


# 4. Brain Cropping

In [17]:
def crop_brain(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    if len(cnts) == 0:
        return cv2.resize(image, (224, 224))
    c = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    cropped = image[y:y+h, x:x+w]
    return cv2.resize(cropped, (224, 224))

# 5. Load Datasets

In [18]:
def load_images(folder, label):
    imgs, lbls = [], []
    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)
        img = cv2.imread(path)
        if img is None:
            continue
        img = crop_brain(img)
        imgs.append(img)
        lbls.append(label)
    return imgs, lbls

def load_dataset(yes_dir, no_dir):
    y_imgs, y_lbls = load_images(yes_dir, 1)
    n_imgs, n_lbls = load_images(no_dir, 0)
    X = np.array(y_imgs + n_imgs, dtype=np.uint8)
    y = np.array(y_lbls + n_lbls, dtype=np.int32)
    return X, y

X_train_raw, y_train = load_dataset(BR35H_YES, BR35H_NO)
X_val_raw, y_val = load_dataset(NAV_YES, NAV_NO)
X_test_raw, y_test = load_dataset(MOS_YES, MOS_NO)

print("Train (Br35H):", X_train_raw.shape, np.bincount(y_train))
print("Val (Navoneel):", X_val_raw.shape, np.bincount(y_val))
print("Test (Mostafa):", X_test_raw.shape, np.bincount(y_test))

Train (Br35H): (3000, 224, 224, 3) [1500 1500]
Val (Navoneel): (253, 224, 224, 3) [ 98 155]
Test (Mostafa): (5450, 224, 224, 3) [2725 2725]


# 6. Data Augmentation & Preprocessing

In [19]:
# 6. Data Augmentation & Preprocessing (UPDATED - NaN SAFE)

BATCH = 32
steps_per_epoch = max(1, len(X_train_raw) // BATCH)
print("Batch:", BATCH, "Steps/epoch:", steps_per_epoch)

# Notes:
# - cv2.imread loads BGR; ResNet preprocess expects RGB (it converts RGB->BGR internally).
# - Random histogram matching is source-only (uses only training images) to randomize intensity styles.
# - This version is NaN/Inf-safe and clips to [0, 255] before ResNet preprocess.

from skimage.exposure import match_histograms

_rng = np.random.default_rng(42)
_ref_idx = _rng.choice(len(X_train_raw), size=min(256, len(X_train_raw)), replace=False)
_ref_pool = X_train_raw[_ref_idx]


def _bgr_to_rgb_f32(x):
    x = x.astype(np.float32)
    return x[..., ::-1]


def _safe_clip_uint8_range(x):
    # Remove NaN/Inf then clip to the intensity range expected by ImageNet preprocess.
    x = np.nan_to_num(x, nan=0.0, posinf=255.0, neginf=0.0)
    return np.clip(x, 0.0, 255.0).astype(np.float32)


def random_histogram_match_rgb(x_rgb: np.ndarray) -> np.ndarray:
    ref = _ref_pool[_rng.integers(0, len(_ref_pool))]
    ref_rgb = _bgr_to_rgb_f32(ref)

    # Histogram match in float32, preserve_range, then sanitize.
    y = match_histograms(x_rgb, ref_rgb, channel_axis=-1)
    y = y.astype(np.float32)
    y = _safe_clip_uint8_range(y)
    return y


def preprocess_train(x):
    # 1) Convert BGR->RGB
    x = _bgr_to_rgb_f32(x)

    # 2) Source-only style randomization via histogram matching
    if _rng.random() < 0.25:
        x = random_histogram_match_rgb(x)

    # 3) Final safety (even if histogram match didn't run)
    x = _safe_clip_uint8_range(x)

    # 4) ResNet preprocessing (expects RGB input)
    x = resnet_preprocess(x)
    return x


datagen = ImageDataGenerator(
    preprocessing_function=preprocess_train,
    rotation_range=15,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.20,
    shear_range=0.10,
    horizontal_flip=True,
    brightness_range=(0.80, 1.20),
    fill_mode="nearest"
)

train_flow = datagen.flow(
    X_train_raw, y_train,
    batch_size=BATCH,
    shuffle=True,
    seed=42
)

# Validation/Test: do NOT histogram-match; just do correct color + ResNet preprocess.
X_val_p  = resnet_preprocess(_safe_clip_uint8_range(X_val_raw[..., ::-1].astype(np.float32)))
X_test_p = resnet_preprocess(_safe_clip_uint8_range(X_test_raw[..., ::-1].astype(np.float32)))

Batch: 32 Steps/epoch: 93


# 7. ResNet50 Binary Classifier

In [ ]:
# 7. ResNet50 Binary Classifier (UPDATED)

with strategy.scope():
    base = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base.trainable = False

    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.6)(x)

    out = layers.Dense(1, activation="sigmoid", dtype="float32")(x)

    model = models.Model(inputs=base.input, outputs=out)

    loss_fn = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=loss_fn,
        metrics=[
            "accuracy",
            tf.keras.metrics.AUC(name="auc")
        ]
    )

model.summary()

# 8. Training

In [21]:
# 8. Training (UPDATED)

callbacks = [
    EarlyStopping(monitor="val_auc", mode="max", patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_auc", mode="max", patience=3, factor=0.2, min_lr=1e-6),
    ModelCheckpoint("best_resnet50_train_br35h.keras", monitor="val_auc", mode="max", save_best_only=True),
    CSVLogger("training_train_br35h.csv", append=False),
    TqdmCallback(verbose=2)
]

print("\n=== Train on Br35H | Validate on Navoneel ===")
history = model.fit(
    train_flow,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_val_p, y_val),
    epochs=30,
    callbacks=callbacks,
    verbose=0
)

0epoch [00:00, ?epoch/s]


=== Train on Br35H | Validate on Navoneel ===


  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

INFO:tensorflow:Collective all_reduce tensors: 6 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1


  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

...

  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

# 9. Evaluate on Mostafa Test

In [22]:
# 9. Evaluate on Mostafa Test (UPDATED - NaN GUARD)

# If something still goes wrong numerically, this will make it obvious.
if not np.isfinite(X_test_p).all():
    raise ValueError("X_test_p contains NaN/Inf")

loss, acc, auc = model.evaluate(X_test_p, y_test, verbose=1)
print("=" * 50)
print("FINAL TEST (Mostafa)")
print("=" * 50)
print(f"Test Accuracy : {acc:.4f}")
print(f"Test AUC      : {auc:.4f}")
print(f"Test Loss     : {loss:.4f}")
print("=" * 50)

171/171 ━━━━━━━━━━━━━━━━━━━━ 11s 64ms/step - accuracy: 0.8268 - auc: 0.4095 - loss: 0.5316
FINAL TEST (Mostafa)
Test Accuracy : 0.7457
Test AUC      : 0.8060
Test Loss     : 0.8363


# Fine Tuning

In [23]:
# Fine Tuning (UPDATED)

with strategy.scope():
    for layer in model.layers:
        layer.trainable = False

    for layer in model.layers:
        if layer.name.startswith("conv5_"):
            layer.trainable = True

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False

    loss_fn = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss=loss_fn,
        metrics=[
            "accuracy",
            tf.keras.metrics.AUC(name="auc")
        ]
    )

callbacks_ft = [
    EarlyStopping(monitor="val_auc", mode="max", patience=4, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_auc", mode="max", patience=2, factor=0.2, min_lr=1e-7),
    ModelCheckpoint("best_resnet50_finetuned_conv5.keras", monitor="val_auc", mode="max", save_best_only=True),
    CSVLogger("training_finetune_conv5.csv", append=False),
    TqdmCallback(verbose=2)
]

print("\n=== Fine-tune conv5_* | Train on Br35H | Validate on Navoneel ===")
history_ft = model.fit(
    train_flow,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_val_p, y_val),
    epochs=15,
    callbacks=callbacks_ft,
    verbose=0
)

0epoch [00:00, ?epoch/s]


=== Fine-tune conv5_* | Train on Br35H | Validate on Navoneel ===


  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

INFO:tensorflow:Collective all_reduce tensors: 20 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1


  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

...

  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

  0%|          | 0.00/93.0 [00:00<?, ?batch/s]

# Re-Test after Fine Tuning

In [24]:
# Re-Test after Fine Tuning (UPDATED)

loss, acc, auc = model.evaluate(X_test_p, y_test, verbose=1)
print("=" * 50)
print("FINAL TEST AFTER FINE-TUNING (Mostafa)")
print("=" * 50)
print(f"Test Accuracy : {acc:.4f}")
print(f"Test AUC      : {auc:.4f}")
print(f"Test Loss     : {loss:.4f}")
print("=" * 50)

171/171 ━━━━━━━━━━━━━━━━━━━━ 11s 64ms/step - accuracy: 0.8560 - auc: 0.4137 - loss: 0.5054
FINAL TEST AFTER FINE-TUNING (Mostafa)
Test Accuracy : 0.7628
Test AUC      : 0.8144
Test Loss     : 0.8889


# 10. Grad-CAM

In [ ]:
def make_gradcam_heatmap(img_batch_preprocessed, model, last_conv_layer_name="conv5_block3_out"):
    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, preds = grad_model(img_batch_preprocessed, training=False)
        class_channel = preds[:, 0]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)

    heatmap = tf.maximum(heatmap, 0)
    heatmap = heatmap / (tf.reduce_max(heatmap) + 1e-8)

    return heatmap.numpy().astype(np.float32)

idx = 0
raw = X_test_raw[idx]
inp = X_test_p[idx:idx+1]

heatmap = make_gradcam_heatmap(inp, model, last_conv_layer_name="conv5_block3_out")
heatmap = cv2.resize(heatmap, (224, 224)).astype(np.float32)

plt.figure(figsize=(5, 5))
plt.imshow(raw)
plt.imshow(heatmap, alpha=0.4, cmap="jet")
plt.axis("off")
plt.show()

# 11. LIME

In [ ]:
lime_explainer = lime_image.LimeImageExplainer()

def predict_fn_lime(images):
    images = images.astype(np.float32)
    images = resnet_preprocess(images)
    p = model.predict(images, verbose=0).ravel()
    return np.vstack([1 - p, p]).T

idx = 0
explanation = lime_explainer.explain_instance(
    X_test_raw[idx].astype("double"),
    predict_fn_lime,
    top_labels=1,
    num_samples=1000
)

temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0],
    positive_only=True,
    num_features=5
)

plt.figure(figsize=(5, 5))
plt.imshow(mark_boundaries(temp / 255.0, mask))
plt.axis("off")
plt.show()

# 12. SHAP

In [ ]:
idx = 0

bg_idx = np.random.choice(X_train_raw.shape[0], 20, replace=False)
background = resnet_preprocess(X_train_raw[bg_idx].astype(np.float32))

x_uint8 = X_test_raw[idx:idx+1].astype(np.uint8)
x_model = resnet_preprocess(x_uint8.astype(np.float32))
x_plot = (x_uint8.astype(np.float32) / 255.0).clip(0, 1)

explainer = shap.GradientExplainer(model, background)
shap_vals = explainer.shap_values(x_model)

if isinstance(shap_vals, list):
    shap_vals = shap_vals[0]
else:
    shap_vals = shap_vals[..., 0] if shap_vals.ndim == 5 else shap_vals

shap.image_plot([shap_vals], x_plot)